## Calculate the probabilities

In [5]:
def calculate_unigram_probs(tokens):
    train_probs = {}

    for i in range(0, len(tokens)):
        if (tokens[i] in list(train_probs.keys())):
            train_probs[tokens[i]] += 1
        else:
            train_probs[tokens[i]] = 1
    
    total = sum(list(train_probs.values()))
    for i in list(train_probs.keys()):
        train_probs[i] = train_probs[i]/total
    
    out = {
        'keys': np.array(list(train_probs.keys())),
        'values': np.array(list(train_probs.values()))
    }
    return out

In [144]:
def calculate_bigram_probs(tokens):
    bigram = {}
    for i in range(0, len(tokens)-1):
        if tokens[i] not in list(bigram.keys()):
            bigram[tokens[i]] = {}
        
        if tokens[i+1] not in list(bigram[tokens[i]].keys()):
            bigram[tokens[i]][tokens[i+1]] = 1
        else:
            bigram[tokens[i]][tokens[i+1]] += 1
        #print(i+1, tokens[i+1])
    
    for k in list(bigram.keys()):
        temp = np.array(list(bigram[k].values()))
        for l in list(bigram[k].keys()):
            bigram[k][l] = bigram[k][l] / temp.sum()
    
    return bigram

In [1]:
def calc_interpolated_probs(unigram, bigram, lambda1, lambda2):
    new_probs = bigram
    uni_keys = list(unigram['keys'])
    uni_values = list(unigram['values'])
    
    for key_i in list(bigram.keys()):
        for key_j in list(bigram[key_i].keys()):
            if (key_j not in uni_keys):
                new_probs[key_i][key_j] = (lambda1*bigram[key_i][key_j])#+ (lambda2*uni_values[uni_keys.index(key_j)])
            else:
                new_probs[key_i][key_j] = (lambda1*bigram[key_i][key_j]) + (lambda2*uni_values[uni_keys.index(key_j)])
    return new_probs

## Suggest words

In [8]:
def suggest_words_unigram(prob_values, prob_keys, word_, n = 5):
    
    subchar_indx = np.char.find(prob_keys, word_)
    count_ = 0
    for i in range(0, subchar_indx.shape[0]):
        if (subchar_indx[i] == 0):
            count_ += 1
    
    sub_idx_keys = np.zeros(count_, '<U23')
    sub_idx_values = np.zeros(count_, 'float64')
    
    count_ = 0
    for i in range(0, subchar_indx.shape[0]):
        if (subchar_indx[i] == 0):
            sub_idx_keys[count_] = prob_keys[i]
            sub_idx_values[count_] = prob_values[i]
            count_ += 1
    
    temp_probs = np.sort(sub_idx_values)[::-1]
    prob_list = list(sub_idx_values)
    
    if(n < len(temp_probs)):
        times = n
    else:
        times = len(temp_probs)
    
    out_list = [0]*times
    out_probs = [0]*times
    
    for i in range(0, times):
        out_list[i] = sub_idx_keys[prob_list.index(temp_probs[i])]
        out_probs[i] = sub_idx_values[prob_list.index(temp_probs[i])]
    
    return out_list, out_probs

In [132]:
def suggest_words_bigram(bigram_probs, word_, n = 5):
    
    words = word_.split(' ')
    words = words[(len(words)-2):(len(words))]
    
    if(len(words) == 0):
        words = [0]*2
        words[0] = "<s>"
        words[1] = ""
    elif (len(words) == 1):
        words.append("<s>")
        words.reverse()
        #print(words)
    #print(words)
    
    g_split = bigram_probs[words[0]]
    
    out_list, out_probs = suggest_words_unigram(np.array(list(g_split.values())), np.array(list(g_split.keys())), words[1], n)
    
    return out_list, out_probs

# getting candidates

In [21]:
def deletions_list(word, num = 1):
    if (num == 1):
        new_list = [0] * len(word)
        for i in range(0, len(word)):
            new_list[i] = word[:i] + word[i+1:]
        return new_list
    elif(num == 2):
        new_list = deletions_list(word, 1)
        list2 = []
        for i in new_list:
            temp = deletions_list(i, 1)
            for a in temp:
                list2.append(a)
        new_list = list(set(list2))
        return new_list
    else:
        return "aisa nahi chaly ga"
    

In [22]:
def substitutions_list (word, letters, num = 1):
    if (num == 1):
        new_list = []
        for i in range(0, len(word)):
            for a in range(0, len(letters)):
                if (letters[a] != word[i]):
                    temp = word[:i] + letters[a] +word[i+1:]
                    new_list.append(temp)
        return new_list
    elif(num == 2):
        temp_list = substitutions_list(word, letters, 1)
        final_list = []
        for i in temp_list:
            temp = substitutions_list(i, letters, 1)
            for a in temp:
                final_list.append(a)
        new_list = []
        for i in final_list:
            if i not in temp_list:
                new_list.append(i)
        return list(set(new_list))
    else:
        return "mazaaak ey?"

In [23]:
def insertions_list(word, letters, num = 1):
    if (num == 1):
        new_list = []
        for i in range(0, len(word)+1):
            if(i != len(word)):
                for a in letters:
                    new_list.append(word[:i] + a +word[i:])
            else:
                for a in letters:
                    new_list.append(word[:i] + a)
        return new_list
    elif (num == 2):
        temp_list = insertions_list(word, letters, 1)
        final_list = []
        for i in temp_list:
            for a in insertions_list(i, letters, 1):
                final_list.append(a)
        return (list(set(final_list)))

In [24]:
def get_candidates(word, letters, dist = 1):
    dels_list = deletions_list(word, dist)
    ins_list = insertions_list(word, letters, dist)
    subs_list = substitutions_list(word, letters, dist)
    candidates = dels_list+ins_list+subs_list
    return candidates

In [25]:
def get_candidates_1n2(word, letters):
    dels_list1 = deletions_list(word, 1)
    ins_list1 = insertions_list(word, letters, 1)
    subs_list1 = substitutions_list(word, letters, 1)
 
    dels_list2 = deletions_list(word, 2)
    ins_list2 = insertions_list(word, letters, 2)
    subs_list2 = substitutions_list(word, letters, 2)
 
    candidates = dels_list1+ins_list1+subs_list1+dels_list2+ins_list2+subs_list2
    return list(set(candidates))

In [26]:
def reduced_candidates(candidates, wordslist):
    final_list = []
    for i in range(0, len(list(candidates))):
        if (candidates[i] in wordslist):
            final_list.append(candidates[i])
    return final_list

In [27]:
def rank_candidates(word1, candidates, unigram_prob, bigram_prob, lambda1=0.7, lambda2=0.3):
    Candids = {}
    uni_keys = list(unigram_prob['keys'])
    uni_values = list(unigram_prob['values'])
    
    for i in range(0, len(candidates)):
        if ((word1 not in list(bigram_prob.keys())) or (candidates[i] not in list(bigram_prob[word1].keys()))):
            if(candidates[i] not in uni_keys):
                Candids[candidates[i]] = 0
            else:
                Candids[candidates[i]] = lambda2*(uni_values[uni_keys.index(candidates[i])])
        else:
            Candids[candidates[i]] = lambda1*(bigram_prob[word1][candidates[i]]) + lambda2*(uni_values[uni_keys.index(candidates[i])])
    
    probs = np.array(list(Candids.values()), 'float64')
    sorted_probs = np.sort(probs)[::-1]
    sorted_candids = []
    
    temp_keys = list(Candids.keys())
    temp_values = list(Candids.values())
    
    for i in range(0, len(sorted_probs)):
        t_idx = temp_values.index(sorted_probs[i])
        sorted_candids.append(str(temp_keys[t_idx]))
        
        temp_keys = temp_keys[:t_idx] + temp_keys[t_idx+1:]
        temp_values = temp_values[:t_idx] + temp_values[t_idx+1:]
    
    return sorted_candids, sorted_probs

In [45]:
def rank10_candidates(word1, candidates, unigram_prob, bigram_prob, lambda1=0.7, lambda2=0.3):
    Candids = {}
    
    uni_keys = list(unigram_prob['keys'])
    uni_values = list(unigram_prob['values'])
    
    for i in range(0, len(candidates)):
        if ((word1 not in list(bigram_prob.keys())) or (candidates[i] not in list(bigram_prob[word1].keys()))):
            if(candidates[i] not in uni_keys):
                Candids[candidates[i]] = 0
            else:
                Candids[candidates[i]] = lambda2*(uni_values[uni_keys.index(candidates[i])])
        else:
            Candids[candidates[i]] = lambda1*(bigram_prob[word1][candidates[i]]) + lambda2*(uni_values[uni_keys.index(candidates[i])])
    
    probs = np.array(list(Candids.values()), 'float64')
    sorted_probs = np.sort(probs)[::-1]
    sorted_candids = []
    
    temp_keys = list(Candids.keys())
    temp_values = list(Candids.values())
    
    if(len(sorted_probs) < 10):
        n = len(sorted_probs)
    else:
        n = 10
    for i in range(0, n):
        t_idx = temp_values.index(sorted_probs[i])
        sorted_candids.append(str(temp_keys[t_idx]))
        
        temp_keys = temp_keys[:t_idx] + temp_keys[t_idx+1:]
        temp_values = temp_values[:t_idx] + temp_values[t_idx+1:]
    
    return sorted_candids, sorted_probs

In [46]:
def get_ranked_candidates(word_, letters, wordslist, unigram_prob, bigram_prob, lambda1=0.7, lambda2=0.3):
    
    words = word_.split(' ')
    words = words[(len(words)-2):(len(words))]
    
    if (len(words) == 1):
        words.append("<s>")
        words.reverse()
    
    if(words[-1] not in wordslist):
        print(words[-1], " is an error word")
        print('calculating its candidate list')
        can1 = get_candidates_1n2(words[-1], letters)
        print('reducing its candidate list')
        can2 = reduced_candidates(can1, wordslist)
        print('ranking candidates')
        can3, probs2 = rank_candidates(words[-2], can2, unigram_prob, bigram_prob, lambda1, lambda2)
        return can3, probs2
    else:
        print(words[-1], " is not an error word")
        return 1, 1

In [47]:
def get_rank10_candidates(word_, letters, wordslist, unigram_prob, bigram_prob, lambda1=0.7, lambda2=0.3):
    
    words = word_.split(' ')
    words = words[(len(words)-2):(len(words))]
    
    if (len(words) == 1):
        words.append("<s>")
        words.reverse()
    
    if(words[-1] not in wordslist):
        new_word_list = []
        for i in range(0, len(wordslist)):
            if((len(wordslist[i]) > (len(words[-1])-3) ) and ((len(wordslist[i]) < (len(words[-1])+3) ))):
                new_word_list.append(wordslist[i])
        
        print(words[-1], " is an error word")
        print('calculating its candidate list')
        can1 = get_candidates_1n2(words[-1], letters)
        print('reducing its candidate list')
        can2 = reduced_candidates(can1, new_word_list)
        print('ranking candidates')
        can3, probs2 = rank10_candidates(words[-2], can2, unigram_prob, bigram_prob, lambda1, lambda2)
        return can3, probs2
    else:
        print(words[-1], " is not an error word")
        return 1, 1